---
title: Sample Set nm 254 Creation
description: "creation of the `dataset_eda` schema and nm = 254 sample for the project"
cdt: '2024-09-06T14:32:38'
project: "dataset_EDA"
---

# Wavelength Subset Selection

Based on *a priori* knowledge, 254 is one of the maxima wavelengths. Therefore the best representative of the dataset, as every sample has a detection at that channel. We should create a subset there. This will be created as a table under the schema `dataset_eda` within the 'wine.db' database.

In [ ]:
con.sql(
"""
CREATE SCHEMA IF NOT EXISTS dataset_eda;
CREATE OR REPLACE TABLE
    dataset_eda.nm_254 AS (
SELECT
    *
FROM
    pbl.chromatogram_spectra_long
WHERE
    wavelength = 254
ORDER BY
    id, idx);
SELECT * FROM dataset_eda.nm_254
"""
).pl().describe()
